In [4]:
import numpy as np
import pandas as pd

In [5]:
fuels=['Biomass', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Fossil Gas', 'Fossil Hard coal', 'Fossil Oil', 'Fossil Oil shale', 'Fossil Peat', 'Geothermal', 'Hydro Pumped Storage', 'Hydro Pumped Storage', 'Hydro Run-of-river and poundage', 'Hydro Water Reservoir', 'Marine', 'Nuclear', 'Other', 'Other renewable', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore']

In [6]:
df=pd.read_csv('temp.csv')

In [7]:
df

,A,B,C
0,n/e,n/e,1
1,n/e,n/e,2
2,n/e,n/e,3
3,n/e,n/e,4
4,n/e,n/e,5
5,n/e,n/e,10
6,n/e,n/e,9
7,n/e,n/e,8
8,n/e,n/e,7
9,n/e,n/e,6


In [13]:
df=df.replace('n/e',0)

In [20]:
df['D']=df.iloc[:,:3].sum(axis=1)

In [16]:
df['D']=df['B'] + df['C']

In [21]:
df

,A,B,C,D
0,0,0,1,1
1,0,0,2,2
2,0,0,3,3
3,0,0,4,4
4,0,0,5,5
5,0,0,10,10
6,0,0,9,9
7,0,0,8,8
8,0,0,7,7
9,0,0,6,6
